In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import boto3
import pandas as pd
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import time
from sklearn.model_selection import GridSearchCV
import time
import boto3
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import FeatureDefinition, FeatureTypeEnum
from sagemaker.session import Session
from botocore.exceptions import ClientError

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Initialize S3 client
s3_client = boto3.client('s3')

# Define bucket and file path
bucket = 'mlo-team4'
file_key = 'original_data/Housing.csv'

# Read the file from S3
obj = s3_client.get_object(Bucket=bucket, Key=file_key)
housing_data = pd.read_csv(obj['Body'])

In [3]:
# Initialize SageMaker session and Feature Store
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_featurestore_runtime_client = sagemaker_session.sagemaker_featurestore_runtime_client
role = get_execution_role()

In [4]:
role

'arn:aws:iam::637423203755:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [5]:
features_required1 = pd.read_csv('s3://mlo-team4/original_data/Housing.csv')

In [6]:
features_required1.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [7]:
features_required1.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7229300521,20141013T000000,231300.0,2,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [8]:
 features_required1.shape[0]

21613

### Drop Irrelevant Features

In [9]:
features_required1['yr_renovated'].value_counts()


0       20699
2014       91
2013       37
2003       36
2005       35
        ...  
1954        1
1951        1
1959        1
1934        1
1944        1
Name: yr_renovated, Length: 70, dtype: int64

#### Since there are 20,699 out of 21,613 rows that have 0 as the value for `yr_renovated` let's drop it. Also dropping other irrelevant columns below.

In [10]:
features_required_new = features_required1.drop(columns=['waterfront', 'view', 'sqft_living15', 'sqft_lot15', 'lat', 'long', 'zipcode', 'yr_renovated'])

### Data Types

In [11]:
# Using the .str method to extract the year (first 4 characters)
features_required_new['date'] = features_required_new['date'].str[:4].astype(int)




In [12]:
print(features_required_new.dtypes)


id                 int64
date               int64
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
dtype: object


## Feature Store Set up

In [13]:
# Function to delete feature group if it exists
def delete_feature_group_if_exists(feature_group_name):
    try:
        sagemaker_client.delete_feature_group(FeatureGroupName=feature_group_name)
        print(f"Deleted feature group: {feature_group_name}")
    except sagemaker_client.exceptions.ResourceNotFound:
        print(f"Feature group {feature_group_name} does not exist")
    except sagemaker_client.exceptions.ResourceInUse:
        print(f"Feature group {feature_group_name} is in use and cannot be deleted immediately")

In [33]:
delete_feature_group_if_exists('housing-feature-group')

Deleted feature group: housing-feature-group


In [20]:
# Select features for ingestion
features_required_new = features_required_new[['number', 'event_time', 'date', 'price', 'bedrooms', 
                                               'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 
                                               'condition', 'grade', 'sqft_above', 'sqft_basement', 
                                               'yr_built']]

In [21]:
# Convert float columns to int
features_required_new['price'] = features_required_new['price'].astype('int64')
features_required_new['bathrooms'] = features_required_new['bathrooms'].astype('int64')
features_required_new['floors'] = features_required_new['floors'].astype('int64')

# Verify the data types
print(features_required_new.dtypes)


number           string
event_time       string
date              int64
price             int64
bedrooms          int64
bathrooms         int64
sqft_living       int64
sqft_lot          int64
floors            int64
condition         int64
grade             int64
sqft_above        int64
sqft_basement     int64
yr_built          int64
dtype: object


In [22]:
features_required_new

,number,event_time,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built
0,0,2024-12-12T02:54:13Z,2014,231300,2,1,1180,5650,1,3,7,1180,0,1955
1,1,2024-12-12T02:54:13Z,2014,538000,3,2,2570,7242,2,3,7,2170,400,1951
2,2,2024-12-12T02:54:13Z,2015,180000,2,1,770,10000,1,3,6,770,0,1933
3,3,2024-12-12T02:54:13Z,2014,604000,4,3,1960,5000,1,5,7,1050,910,1965
4,4,2024-12-12T02:54:13Z,2015,510000,3,2,1680,8080,1,3,8,1680,0,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,21608,2024-12-12T02:54:13Z,2014,360000,3,2,1530,1131,3,3,8,1530,0,2009
21609,21609,2024-12-12T02:54:13Z,2015,400000,4,2,2310,5813,2,3,8,2310,0,2014
21610,21610,2024-12-12T02:54:13Z,2014,402101,2,0,1020,1350,2,3,7,1020,0,2009
21611,21611,2024-12-12T02:54:13Z,2015,400000,3,2,1600,2388,2,3,8,1600,0,2004


## Step 1: Split the Data

- 20,000 rows for initial training.
- 1,613 rows for simulation.



In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
# loading data
data = features_required_new

In [25]:
data.shape[0]

21613

In [28]:
# Split to get 3,000 rows for test_data and the rest for simulation_data
test_data, simulation_data = train_test_split(data,
                                            train_size=3000,  # Using train_size to get 3,000 for test
                                            random_state=42)

# Verify the sizes of the splits
print(f"Test data: {test_data.shape[0]} rows")
print(f"Simulation data: {simulation_data.shape[0]} rows")

Test data: 3000 rows
Simulation data: 18613 rows


In [29]:
# First split simulation_data into 10k and the rest
sim_1, temp = train_test_split(simulation_data,
                              train_size=10000,
                              random_state=42)

# Then split the remaining data (8,613 rows) into two roughly equal parts
sim_2, sim_3 = train_test_split(temp,
                               train_size=0.5,  # This will give roughly half of 8,613
                               random_state=42)

# Verify the sizes
print(f"Simulation set 1: {sim_1.shape[0]} rows")
print(f"Simulation set 2: {sim_2.shape[0]} rows")
print(f"Simulation set 3: {sim_3.shape[0]} rows")
print(f"Total rows: {sim_1.shape[0] + sim_2.shape[0] + sim_3.shape[0]}")

Simulation set 1: 10000 rows
Simulation set 2: 4306 rows
Simulation set 3: 4307 rows
Total rows: 18613


In [44]:
sim_1

,number,event_time,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built
14801,14801,2024-12-12T02:54:13Z,2014,513000,3,2,1500,1312,3,3,8,1500,0,2007
16862,16862,2024-12-12T02:54:13Z,2014,1200000,4,3,4740,172497,2,3,11,4740,0,2003
18124,18124,2024-12-12T02:54:13Z,2014,253400,3,2,1400,8640,1,5,7,1400,0,1968
1820,1820,2024-12-12T02:54:13Z,2014,275000,2,1,1180,6552,1,4,6,1180,0,1949
13300,13300,2024-12-12T02:54:13Z,2014,328000,4,1,1360,8136,1,3,7,1360,0,1948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13073,13073,2024-12-12T02:54:13Z,2015,469000,4,2,2240,5624,1,3,7,1520,720,1961
19941,19941,2024-12-12T02:54:13Z,2014,450000,5,2,2710,6220,2,3,8,2710,0,2014
13015,13015,2024-12-12T02:54:13Z,2015,295000,4,2,1650,7200,1,3,7,1650,0,1964
7417,7417,2024-12-12T02:54:13Z,2015,459000,4,3,1900,9077,2,3,7,1900,0,1954


### Ingest Data into Different Feature Groups

### Simulation Data

In [34]:
feature_group_name_simulation = 'housing-feature-group-simulation'
feature_group_name_test = 'housing-feature-group-test'

In [46]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import FeatureDefinition, FeatureTypeEnum

# Define training feature group
feature_group_name_simulation = 'housing-feature-group-simulation'

feature_definitions_simulation = FeatureGroup(
    name=feature_group_name_simulation,
    sagemaker_session=sagemaker_session,
    feature_definitions=[
        FeatureDefinition(feature_name="number", feature_type=FeatureTypeEnum.STRING),
        FeatureDefinition(feature_name="event_time", feature_type=FeatureTypeEnum.STRING),
        FeatureDefinition(feature_name="date", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="price", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="bedrooms", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="bathrooms", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_living", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_lot", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="floors", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="condition", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="grade", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_above", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_basement", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="yr_built", feature_type=FeatureTypeEnum.FRACTIONAL)
    ]
)

In [47]:
# Create the training feature group
feature_definitions_simulation.create(
    record_identifier_name="number",
    event_time_feature_name="event_time",
    s3_uri=f's3://mlo-team4/features/simulation/',
    role_arn=role,
    enable_online_store=True
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:637423203755:feature-group/housing-feature-group-simulation',
 'ResponseMetadata': {'RequestId': 'a161076d-14ed-4bdf-9a8c-c004f8063061',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a161076d-14ed-4bdf-9a8c-c004f8063061',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Thu, 12 Dec 2024 03:10:23 GMT'},
  'RetryAttempts': 0}}

In [48]:
# Ingest training data
feature_definitions_simulation.ingest(data_frame=sim_1, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='housing-feature-group-simulation', feature_definitions={'number': {'FeatureName': 'number', 'FeatureType': 'String'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'date': {'FeatureName': 'date', 'FeatureType': 'Fractional'}, 'price': {'FeatureName': 'price', 'FeatureType': 'Fractional'}, 'bedrooms': {'FeatureName': 'bedrooms', 'FeatureType': 'Fractional'}, 'bathrooms': {'FeatureName': 'bathrooms', 'FeatureType': 'Fractional'}, 'sqft_living': {'FeatureName': 'sqft_living', 'FeatureType': 'Fractional'}, 'sqft_lot': {'FeatureName': 'sqft_lot', 'FeatureType': 'Fractional'}, 'floors': {'FeatureName': 'floors', 'FeatureType': 'Fractional'}, 'condition': {'FeatureName': 'condition', 'FeatureType': 'Fractional'}, 'grade': {'FeatureName': 'grade', 'FeatureType': 'Fractional'}, 'sqft_above': {'FeatureName': 'sqft_above', 'FeatureType': 'Fractional'}, 'sqft_basement': {'FeatureName': 'sqft_basement', 'FeatureType': 'Fractional'},

### Testing Data

In [39]:
# Define test feature group
feature_group_name_test = 'housing-feature-group-test'

feature_definitions_test = FeatureGroup(
    name=feature_group_name_test,
    sagemaker_session=sagemaker_session,
    feature_definitions=[
        FeatureDefinition(feature_name="number", feature_type=FeatureTypeEnum.STRING),
        FeatureDefinition(feature_name="event_time", feature_type=FeatureTypeEnum.STRING),
        FeatureDefinition(feature_name="date", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="price", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="bedrooms", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="bathrooms", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_living", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_lot", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="floors", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="condition", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="grade", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_above", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="sqft_basement", feature_type=FeatureTypeEnum.FRACTIONAL),
        FeatureDefinition(feature_name="yr_built", feature_type=FeatureTypeEnum.FRACTIONAL),
    ]
)


In [40]:

# Create the test feature group
feature_definitions_test.create(
    record_identifier_name="number",
    event_time_feature_name="event_time",
    s3_uri=f's3://mlo-team4/features/test/',
    role_arn=role,
    enable_online_store=True
)


{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:637423203755:feature-group/housing-feature-group-test',
 'ResponseMetadata': {'RequestId': '8d2b5c50-144d-4f98-add7-7c7d8867888f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d2b5c50-144d-4f98-add7-7c7d8867888f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Thu, 12 Dec 2024 03:02:34 GMT'},
  'RetryAttempts': 0}}

In [43]:
# Ingest test data
feature_definitions_test.ingest(data_frame=test_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='housing-feature-group-test', feature_definitions={'number': {'FeatureName': 'number', 'FeatureType': 'String'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'date': {'FeatureName': 'date', 'FeatureType': 'Fractional'}, 'price': {'FeatureName': 'price', 'FeatureType': 'Fractional'}, 'bedrooms': {'FeatureName': 'bedrooms', 'FeatureType': 'Fractional'}, 'bathrooms': {'FeatureName': 'bathrooms', 'FeatureType': 'Fractional'}, 'sqft_living': {'FeatureName': 'sqft_living', 'FeatureType': 'Fractional'}, 'sqft_lot': {'FeatureName': 'sqft_lot', 'FeatureType': 'Fractional'}, 'floors': {'FeatureName': 'floors', 'FeatureType': 'Fractional'}, 'condition': {'FeatureName': 'condition', 'FeatureType': 'Fractional'}, 'grade': {'FeatureName': 'grade', 'FeatureType': 'Fractional'}, 'sqft_above': {'FeatureName': 'sqft_above', 'FeatureType': 'Fractional'}, 'sqft_basement': {'FeatureName': 'sqft_basement', 'FeatureType': 'Fractional'}, 'yr_b

### Store the rest of the simulation data into bucket for testing

In [ ]:
simulation_2 = sim_2.astype(str)

In [72]:
simulation_3 = sim_3.astype(str)


In [73]:
simulation_3.dtypes


number           object
event_time       object
date             object
price            object
bedrooms         object
bathrooms        object
sqft_living      object
sqft_lot         object
floors           object
condition        object
grade            object
sqft_above       object
sqft_basement    object
yr_built         object
dtype: object

In [74]:
import boto3
import os

# S3 bucket details
bucket_name = "mlo-team4"
s3_prefix = "housing-streaming-data"  # Folder name in the S3 bucket

# Save local CSVs to a temp directory before uploading
output_dir = "./feature_group_csvs"
os.makedirs(output_dir, exist_ok=True)

# File paths
train_csv_path = os.path.join(output_dir, "simulation_3.csv")

# Save DataFrames as CSV
sim_2.to_csv(train_csv_path, index=False)


# Upload to S3
s3_client = boto3.client("s3")

def upload_to_s3(local_path, s3_bucket, s3_key):
    s3_client.upload_file(local_path, s3_bucket, s3_key)
    print(f"Uploaded {local_path} to s3://{s3_bucket}/{s3_key}")

# Upload files
upload_to_s3(train_csv_path, bucket_name, f"{s3_prefix}/simulation_3.csv")



Uploaded ./feature_group_csvs/simulation_3.csv to s3://mlo-team4/housing-streaming-data/simulation_3.csv


In [ ]:
# S3 bucket details
bucket_name = "mlo-team4"
s3_prefix = "housing-streaming-data"  # Folder name in the S3 bucket

# Save local CSVs to a temp directory before uploading
output_dir = "./feature_group_csvs"
os.makedirs(output_dir, exist_ok=True)

# File paths
train_csv_path = os.path.join(output_dir, "simulation_2.csv")

# Save DataFrames as CSV
sim_2.to_csv(train_csv_path, index=False)


# Upload to S3
s3_client = boto3.client("s3")

def upload_to_s3(local_path, s3_bucket, s3_key):
    s3_client.upload_file(local_path, s3_bucket, s3_key)
    print(f"Uploaded {local_path} to s3://{s3_bucket}/{s3_key}")

# Upload files
upload_to_s3(train_csv_path, bucket_name, f"{s3_prefix}/simulation_2.csv")

In [37]:
# File path for simulation data
simulation_csv_path = os.path.join(output_dir, "simulation_data.csv")

# Save simulation data as CSV
simulation_data.to_csv(simulation_csv_path, index=False)

# Upload simulation data to S3
upload_to_s3(simulation_csv_path, bucket_name, f"{s3_prefix}/simulation_data.csv")

print("All datasets have been saved and uploaded to S3.")

Uploaded ./feature_group_csvs/simulation_data.csv to s3://mlo-team4/feature_store_splits/simulation_data.csv
All datasets have been saved and uploaded to S3.


### Example: Function to call all parquets from feature store

In [73]:
import boto3
import pandas as pd
import awswrangler as wr
from typing import List

def list_parquet_files(bucket: str, prefix: str) -> List[str]:
    """
    List all parquet files under a given S3 prefix, including date-partitioned folders
    
    Args:
        bucket (str): S3 bucket name
        prefix (str): S3 prefix path
        
    Returns:
        List[str]: List of complete S3 URIs for parquet files
    """
    s3_client = boto3.client('s3')
    parquet_files = []
    
    # Remove s3:// if present in bucket name
    bucket = bucket.replace('s3://', '')
    
    # Use delimiter to list folders first
    paginator = s3_client.get_paginator('list_objects_v2')
    
    # First, get all year folders
    year_pages = paginator.paginate(Bucket=bucket, Prefix=prefix, Delimiter='/')
    
    for year_page in year_pages:
        if 'CommonPrefixes' in year_page:
            for year_prefix in year_page['CommonPrefixes']:
                year_path = year_prefix['Prefix']
                
                # For each year, get month folders
                month_pages = paginator.paginate(Bucket=bucket, Prefix=year_path, Delimiter='/')
                for month_page in month_pages:
                    if 'CommonPrefixes' in month_page:
                        for month_prefix in month_page['CommonPrefixes']:
                            month_path = month_prefix['Prefix']
                            
                            # For each month, get day folders
                            day_pages = paginator.paginate(Bucket=bucket, Prefix=month_path, Delimiter='/')
                            for day_page in day_pages:
                                if 'CommonPrefixes' in day_page:
                                    for day_prefix in day_page['CommonPrefixes']:
                                        day_path = day_prefix['Prefix']
                                        
                                        # Finally, list parquet files in the day folder
                                        file_pages = paginator.paginate(Bucket=bucket, Prefix=day_path)
                                        for file_page in file_pages:
                                            if 'Contents' in file_page:
                                                for obj in file_page['Contents']:
                                                    if obj['Key'].endswith('.parquet'):
                                                        parquet_files.append(f"s3://{bucket}/{obj['Key']}")
    
    return parquet_files

def read_all_parquets(s3_path: str) -> pd.DataFrame:
    """
    Read and concatenate all parquet files from a given S3 path with date partitions
    
    Args:
        s3_path (str): Base S3 path including bucket and prefix
        
    Returns:
        pd.DataFrame: Combined DataFrame from all parquet files
    """
    # Split S3 path into bucket and prefix
    s3_path = s3_path.rstrip('/')
    bucket = s3_path.split('/')[2]
    prefix = '/'.join(s3_path.split('/')[3:]) + '/'
    
    # Get list of all parquet files
    print("Listing parquet files...")
    parquet_files = list_parquet_files(bucket, prefix)
    
    if not parquet_files:
        raise ValueError(f"No parquet files found in {s3_path}")
    
    # Read all parquet files
    print(f"Found {len(parquet_files)} parquet files")
    print("Reading parquet files...")
    df = wr.s3.read_parquet(path=parquet_files)
    
    return df

# Example usage
try:
    s3_path = "s3://mlo-team4/features/simulation/637423203755/sagemaker/us-east-2/offline-store/housing-feature-group-simulation-1733973023/data/"
    combined_df = read_all_parquets(s3_path)
    print(f"Combined DataFrame shape: {combined_df.shape}")
except Exception as e:
    print(f"Error: {str(e)}")

Listing parquet files...
Found 42 parquet files
Reading parquet files...
Combined DataFrame shape: (10129, 17)


In [72]:
combined_df

,number,event_time,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built,write_time,api_invocation_time,is_deleted
0,9837,2024-12-09T21:54:16Z,2014.0,491000.0,3.0,2.00,1660.0,5070.0,2.0,3.0,8.0,1660.0,0.0,1950.0,2024-12-09 22:00:56.560000+00:00,2024-12-09 21:55:58+00:00,False
1,10320,2024-12-09T21:54:16Z,2015.0,365000.0,4.0,1.00,1760.0,8906.0,1.0,3.0,7.0,1230.0,530.0,1966.0,2024-12-09 22:00:56.560000+00:00,2024-12-09 21:55:58+00:00,False
2,1913,2024-12-09T21:54:16Z,2014.0,315000.0,3.0,1.75,1660.0,8160.0,1.0,4.0,7.0,1660.0,0.0,1951.0,2024-12-09 22:00:56.560000+00:00,2024-12-09 21:55:58+00:00,False
3,6058,2024-12-09T21:54:16Z,2014.0,530000.0,2.0,1.50,1390.0,5000.0,1.0,4.0,8.0,970.0,420.0,1954.0,2024-12-09 22:00:56.560000+00:00,2024-12-09 21:55:58+00:00,False
4,18577,2024-12-09T21:54:16Z,2014.0,240000.0,3.0,2.25,1410.0,7290.0,1.0,3.0,7.0,940.0,470.0,1980.0,2024-12-09 22:00:56.560000+00:00,2024-12-09 21:55:58+00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,924,2024-12-11T06:36:06Z,2014.0,340000.0,3.0,1.00,1250.0,4800.0,1.0,4.0,7.0,1250.0,0.0,1951.0,2024-12-11 06:41:03.094000+00:00,2024-12-11 06:36:32+00:00,False
3738,949,2024-12-11T06:36:06Z,2014.0,527500.0,2.0,1.00,1170.0,3000.0,1.5,3.0,7.0,1170.0,0.0,1910.0,2024-12-11 06:41:03.094000+00:00,2024-12-11 06:36:33+00:00,False
3739,965,2024-12-11T06:36:06Z,2014.0,534950.0,5.0,1.50,2240.0,6337.0,1.0,3.0,8.0,1280.0,960.0,1956.0,2024-12-11 06:41:03.094000+00:00,2024-12-11 06:36:33+00:00,False
3740,972,2024-12-11T06:36:06Z,2014.0,599000.0,2.0,1.00,1550.0,7713.0,1.0,3.0,7.0,1550.0,0.0,1930.0,2024-12-11 06:41:03.094000+00:00,2024-12-11 06:36:33+00:00,False
